In [1]:
# result: address, holdings, previousTradeTimestamp, boughtTotal, holdingCurrently

import pandas as pd
import alchemyHelper
import graphqlHelper
import json

# GLOBAL VARIABLES
uniswapV2Router = "0x7a250d5630b4cf539739df2c5dacb4c659f2488d"
uniswapV3Router = "0x68b3465833fb72A70ecDF485E0e4C7bD8665Fc45"
weiDivisible = 1000000000000000000

# getting count
with open("mainCOUNT.txt", "r") as f:
    rowNum = int(f.read())
count = rowNum + 1

walletsList = pd.read_csv("wallets_test.csv")
walletsList = walletsList.drop_duplicates()


for i, row in walletsList[rowNum:].iterrows():
    bigDICT = {}
    wallet = row["Address"]

    # BUY SECTION
    buyList = alchemyHelper.getBuys(wallet)
    for dict in buyList:
        txnHash = dict["hash"]
        contract = dict["rawContract"]["address"]

        if contract not in bigDICT:
            bigDICT[contract] = {
                "buys": [],
                "sells": []
            }

        bigDICT[contract]['buys'].append(txnHash)

    # SELL SECTION
    sellList = alchemyHelper.getSells(wallet)
    for dict in sellList:
        txnHash = dict["hash"]
        contract = dict["rawContract"]["address"]

        if contract not in bigDICT:
            bigDICT[contract] = {
                "buys": [],
                "sells": []
            }

        bigDICT[contract]['sells'].append(txnHash)

    stillHoldingOrDusted = 0
    sellingIncrementally = 0
    accountTransferORNULL = 0
    stablecoinTransfer = 0
    buysTOTAL = []
    sellsTOTAL = []
    profits = {}

    # bigDICT[contract]
    # ITERATING BIG DICTIONARY
    for contract in bigDICT:
        # REMOVING DUPLICATES
        bigDICT[contract]["buys"] = [*set(bigDICT[contract]["buys"])]
        bigDICT[contract]["sells"] = [*set(bigDICT[contract]["sells"])]

        # CASE0 - IRREGULARITY
        if (bigDICT[contract]["buys"] == []):
            continue

        # CASE1 - still holding token or rugged/dusted
        if (bigDICT[contract]["sells"] == []):
            stillHoldingOrDusted += 1
            continue

        # CASE2 - Great buy and selling incrementally, smart trader
        buyLen = len(bigDICT[contract]["buys"])
        sellLen = len(bigDICT[contract]["sells"])
        diffLen = sellLen / float(buyLen)
        if (diffLen > 2):
            sellingIncrementally += 1

        excludeThisContract = False
        buys = 0.0
        # CHECKING BUYS GRAPHQL
        for buy in bigDICT[contract]["buys"]:
            result = graphqlHelper.getData(buy)
            if (result == -1):
                accountTransferORNULL += 1
                excludeThisContract = True
            elif (result == -2):
                stablecoinTransfer += 1
            else:
                buys += result

        sells = 0.0
        # CHECKING SELLS GRAPHQL
        for sell in bigDICT[contract]["sells"]:
            result = graphqlHelper.getData(sell)
            if (result == -1):
                accountTransferORNULL += 1
                excludeThisContract = True
            elif (result == -2):
                stablecoinTransfer += 1
            else:
                sells += result

        profit = sells - buys
        # STABLECOIN OR JUST TRANSFER
        if (profit == 0.0 or excludeThisContract):
            continue

        buysTOTAL.append(buys)
        sellsTOTAL.append(sells)
        profits[contract] = profit

    with open("test2.json", "w") as file:
        json.dump(profits, file)


"""
with open("test.json", "w") as file:
    json.dump(bigDICT, file)

Data Structure:
>>> d = {}
>>> d['dict1'] = {}
>>> d['dict1']['innerkey'] = 'value'
>>> d['dict1']['innerkey2'] = 'value2'
>>> d

{
    contractAddress: {
        buys: [txn1, txn2, ...],
        sells: [txn1, txn2, ...]
    }
}
"""


'\nwith open("test.json", "w") as file:\n    json.dump(bigDICT, file)\n\nData Structure:\n>>> d = {}\n>>> d[\'dict1\'] = {}\n>>> d[\'dict1\'][\'innerkey\'] = \'value\'\n>>> d[\'dict1\'][\'innerkey2\'] = \'value2\'\n>>> d\n\n{\n    contractAddress: {\n        buys: [txn1, txn2, ...],\n        sells: [txn1, txn2, ...]\n    }\n}\n'